# Virtual Piano Tutor

## Project Description

This is a notebook for tracking my progress on VPT...

- Best Classifier as of 11/30
    - SVM {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

## TODO List

- TODAY
    - Decide on RDF model to keep for rest of project
    - Work on RDF data and annotations
    - Add results to file
    - Rewrite RDF for GridSearchCV
        - Extend RDF
    - Work on ideas for paper
        - Visualizations
    - Play with CAE
    - How to automate this...
    - Windowing/Summarizing
    
- DONE
    - ~~Organize RDF data~~
    - ~~Generate data from already extracted hands...~~
    - ~~Get notebook running on Compute Canada~~
    - ~~Get data on Compute Canada~~
    - ~~Setup CAE to deal with hand images~~
    - ~~setup data for training autoencoder on LH and RH~~
    - ~~Train Autoencoder for LH and Rh~~
    

- Bad Segmentation
    - p3c - left hand (not terrible)
    - p1s - right hand (shouldn't use)
    - p5a - Both could use some work but still caputures most of the left hand (RH not so good...)
    - p5c - not good (left hand passable...)
    
- Add noise to CAE
    - http://scikit-image.org/docs/dev/api/skimage.util.html#random-noise
    
- ~~Multiple Participants~~
    - ~~have one holdout set participant~~
        - ~~Test with p1&2 training p3 testing, then p1&3...~~
    - ~~have one holdout set exercise~~

- Test with RH too

- Windowing data
    - Summarize data for classification
    - Majority Voting (or with probabilities)

- Look for other features
    - Others??
    - ~~Autoencoder features~~
    - ~~HONV~~
    
- Work on hand segmentation
   - See p1e for bad examples
   - How to validate segmentation?
       - Statistical analysis on length and width ratios
       
- Visualize !!!
    - Input 
    - Results !!!
        - F Scores
        - Accuracy
        - Try weighted instead of macro




- Finish Project Description

- ~~Turn into functions~~
    
- ~~Verify Segmentation~~
    - have only done basic verification
    
- ~~FIRST THING: Test by ignoring training data (p1s) and then using train_test_split on recordings~~
    - ~~Data should be ready for spliting~~
    
- ~~Remove data from testing to find culprit~~
    
- ~~Track my progress better !!! (duh through notebooks!)~~

# Setup

## Libraries

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s
import vpt.hand_detection.depth_context_features as dcf

%load_ext autoreload
%autoreload 2

## Some helper functions

### Visualize Hands

In [ ]:
def show_hand(hand):
    dmap = hand.get_original()
    mask = hand.get_mask()
        
    img = (ip.normalize(dmap)*255).astype('uint8')
    img_hand = cv2.bitwise_and(img, img, mask=mask)

    cv2.imshow("OG", img)
    cv2.imshow("Mask", mask)
    cv2.imshow("Hand Only", img_hand)
    
    return cv2.waitKey(20)

### Generate Dataset

In [2]:
def init_hand_generator(folders, testing_p, annotation_file, offset_gen, feature_gen, 
                       M, radius, n_samples=750, ftype=".bin"):
    
    annotations = load_annotations(annotation_file)
    fs = FileStream(folders, ftype, annotations=annotations, ignore=True)

    # generate or load model
    base_model_folder = "data/rdf/trainedmodels/"
    seg_model_path = os.path.join(base_model_folder, "{:s}_M{:d}_rad{:0.2f}".format("mixed_no_{}".format(testing_p), M, radius))
    rdf_hs = load_hs_model("RDF Model", offset_gen, feature_gen, M, radius, n_samples, refresh=False, segmentation_model_path=seg_model_path)

    hd = HandDetector(rdf_hs)
    hg = HandGenerator(fs, hd, annotations)
    
    return hg

In [3]:
def generate_data(hg, feature_type="hog"):

    X_lh = []
    y_lh = []

    X_rh = []
    y_rh = []

    filenames = []
    
    percent = .01

    hgen = hg.hand_generator(debug=True)
    hg_size = hg.size()
    for i, (lh, rh) in enumerate(hgen):
        if lh.label() != None and rh.label() != None:

            filenames.append(lh.get_fpath())
            y_lh.append(lh.label())
            X_lh.append(extract_features(lh.get_hand_img(), feature_type))
            
            y_rh.append(rh.label())
            X_rh.append(extract_features(rh.get_hand_img(), feature_type))
            
        else:
            raise RuntimeWarning("Warning: No label found for hands")
        
        if i / hg_size > percent:
            print("#", end="")
            percent += .01
                

    X_lh = np.array(X_lh)
    y_lh = np.array(y_lh)
    X_rh = np.array(X_rh)
    y_rh = np.array(y_rh)
    filenames = np.array(filenames)
    
    return X_lh, y_lh, X_rh, y_rh, filenames

### Load/Save Data Set

In [4]:
def save_data(X_lh, y_lh, X_rh, y_rh, filenames, testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    
    data_path = os.path.join(base, "without_{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    np.savez_compressed(data_path + data_type + "_data.npz" , X_lh=X_lh, y_lh=y_lh, X_rh=X_rh, y_rh=y_rh, filenames=filenames)

In [5]:
def load_data(testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    data_path = os.path.join(base, "without_{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    data = np.load(data_path + data_type + "_data.npz")    
    return data

## Project Setup

In [6]:
## Some General Parameters
s.participant = "all"
s.sensor = "realsense"

participants = ["p1", "p2", "p3", "p4", "p6"]
posture_folders = {p : os.path.join("data/posture", p) for p in participants}

ftype = ".bin"
folders = "data/posture/"

In [7]:
## Model Parameters
refreshHD = False
refreshCLF = True

## RDF Parameters
M = 5
radius = .15

## Posture Detection Parameters
feature_type = "shog"
annotation_file = "data/posture/annotations.txt"

offset_gen = dcf.generate_feature_offsets
feature_gen = dcf.calc_features

### Generate or Load Data

In [ ]:
#### Generate and Save data for all testing participants
for testing_p in participants:
    
    training_folders = [folder for p, folder in posture_folders.items() if p != testing_p]
    testing_folder = posture_folders[testing_p]
    
    print(training_folders)
    hg_train = init_hand_generator(training_folders, testing_p, annotation_file, offset_gen, feature_gen, 
                       M, radius, n_samples=750, ftype=".bin")
    hg_test = init_hand_generator(testing_folder, testing_p, annotation_file, offset_gen, feature_gen, 
                       M, radius, n_samples=750, ftype=".bin")
    
    print("\n###Generating Training Data | Testing P: {}".format(testing_p))
    X_lh, y_lh, X_rh, y_rh, filenames = generate_data(hg_train, feature_type=feature_type)
    save_data(X_lh, y_lh, X_rh, y_rh, filenames, testing_p, M, radius, feature_type, data_type="train")
    
    print("\n###Generating Testing Data | Testing P: {}".format(testing_p))
    X_lh_test, y_lh_test, X_rh_test, y_rh_test, filenames_test = generate_data(hg_test, feature_type=feature_type)
    save_data(X_lh_test, y_lh_test, X_rh_test, y_rh_test, filenames_test, testing_p, M, radius, feature_type, data_type="test")

In [8]:
#### Load data for a single paricipant
train_data = load_data("p3", M, radius, feature_type, "train")
test_data = load_data("p3", M, radius, feature_type, "test")
print("X LH Train", train_data["X_lh"].shape, "y LH", train_data["y_lh"].shape)
print("X RH Train", train_data["X_rh"].shape, "y RH", train_data["y_rh"].shape)
print("Filenames Train", train_data["filenames"].shape)
print()
print("X LH Test", test_data["X_lh"].shape, "y LH", test_data["y_lh"].shape)
print("X RH Test", test_data["X_rh"].shape, "y RH", test_data["y_rh"].shape)
print("Filenames Test", test_data["filenames"].shape)

X LH Train (15195, 160) y LH (15195,)
X RH Train (15195, 160) y RH (15195,)
Filenames Train (15195,)

X LH Test (4578, 160) y LH (4578,)
X RH Test (4578, 160) y RH (4578,)
Filenames Test (4578,)


In [ ]:
# X_lh = np.concatenate((train_data["X_lh"], test_data["X_lh"]))
# y_lh = np.concatenate((train_data["y_lh"], test_data["y_lh"]))
# X_rh = np.concatenate((train_data["X_rh"], test_data["X_rh"]))
# y_rh = np.concatenate((train_data["y_rh"], test_data["y_rh"]))
# filenames = np.concatenate((train_data["filenames"], test_data["filenames"]))

# print(X_lh.shape, y_lh.shape)
# print(X_rh.shape, y_rh.shape)
# print(filenames.shape)

In [9]:
X_lh = train_data["X_lh"]
y_lh = train_data["y_lh"]
X_rh = train_data["X_rh"]
y_rh = train_data["y_rh"]
filenames = train_data["filenames"]

print("Cross Val Data:")
print(X_lh.shape, y_lh.shape)
print(X_rh.shape, y_rh.shape)
print(filenames.shape)

X_lh_test = test_data["X_lh"]
y_lh_test = test_data["y_lh"]
X_rh_test = test_data["X_rh"]
y_rh_test = test_data["y_rh"]
filenames_test = test_data["filenames"]

print("Validation Data")
print(X_lh_test.shape, y_lh_test.shape)
print(X_rh_test.shape, y_rh_test.shape)
print(filenames_test.shape)

Cross Val Data:
(15195, 160) (15195,)
(15195, 160) (15195,)
(15195,)
Validation Data
(4578, 160) (4578,)
(4578, 160) (4578,)
(4578,)


In [10]:
groups = np.zeros_like(filenames, dtype=int)

for p in ["p1", "p2", "p4", "p6"]:
    p_num = int(p[1])
    groups[np.where(np.char.find(filenames, p) != -1)] = p_num
    
print(groups.shape)
print(np.unique(groups))
print(groups)

(15195,)
[1 2 4 6]
[1 1 1 ..., 6 6 6]


# Classification

### Libraries

In [11]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, LeaveOneGroupOut
from imblearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE

## Load Data for Classification

In [12]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)
rem_static_test = vmatch(filenames_test)

X_lh_train, y_lh_train, filenames_train, groups_train = X_lh[~rem_static], y_lh[~rem_static], filenames[~rem_static], groups[~rem_static]
X_rh_train, y_rh_train, filenames_train, groups_train = X_rh[~rem_static], y_rh[~rem_static], filenames[~rem_static], groups[~rem_static]

In [13]:
X_lh_val, y_lh_val, filenames_val = X_lh_test[~rem_static_test], y_lh_test[~rem_static_test], filenames_test[~rem_static_test]
X_rh_val, y_rh_val, filenames_val = X_rh_test[~rem_static_test], y_rh_test[~rem_static_test], filenames_test[~rem_static_test]

In [14]:
print("Cross Validation Data")
print(X_lh_train.shape, y_lh_train.shape)
print(X_rh_train.shape, y_rh_train.shape)
print(filenames_train.shape)
print(groups_train.shape)

print("Validation Data")
print(X_lh_val.shape, y_lh_val.shape)
print(X_rh_val.shape, y_rh_val.shape)
print(filenames_val.shape)

Cross Validation Data
(13577, 160) (13577,)
(13577, 160) (13577,)
(13577,)
(13577,)
Validation Data
(2650, 160) (2650,)
(2650, 160) (2650,)
(2650,)


## Model Testing

### SVM

In [15]:
steps = [('SMOTE', SMOTE()), ("SVC", SVC())]
pipeline = Pipeline(steps)

## Parameters for SVMs
param_grid = [
  {'SVC__C': [.1, 1, 10, 100], 'SVC__gamma': [.001, .01, .1, 1], 'SVC__kernel': ['rbf', 'linear'], 'SMOTE__kind': ['borderline1', 'borderline2', 'svm']},
 ]

scores = ['accuracy']

logo = LeaveOneGroupOut()

In [ ]:
# Hyper Parameter Tuning
for score in scores:
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print("## Tuning hyper-parameters for {}".format(score))
        print()

        if score is "accuracy":
            scoring = score
        else:
            scoring = '{}_macro'.format(score)
    
        #### TRAIN LH
        clf_lh = GridSearchCV(pipeline, param_grid, cv=logo.split(X_lh_train, y_lh_train, groups=groups_train), scoring=scoring, n_jobs=2)
        clf_lh.fit(X_lh_train, y_lh_train)

        print("Best LH Parameters set found on data set:")
        print()
        print(clf_lh.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_lh.cv_results_['mean_test_score']
        stds  = clf_lh.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_lh.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()
        print()
        
        
        #### TRAIN RH
        clf_rh = GridSearchCV(pipeline, param_grid, cv=logo.split(X_rh_train, y_rh_train, groups=groups_train), scoring=scoring, n_jobs=2)
        clf_rh.fit(X_rh_train, y_rh_train)

        print("Best RH Parameters set found on data set:")
        print()
        print(clf_rh.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_rh.cv_results_['mean_test_score']
        stds  = clf_rh.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_rh.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()

## Tuning hyper-parameters for accuracy



In [ ]:
from sklearn.metrics import accuracy_score
y_true, y_pred = y_lh_val, clf_lh.predict(X_lh_val)
print("Validatation Score:", accuracy_score(y_true, y_pred))
print("Confustion Matrix:\n", confusion_matrix(y_true, y_pred))